# Michelin Stars: Data Cleaning, Extraction, and Transformation

In this part of the project, we will be cleaning and transforming the Michelin Star Data. We obtained the data in three seperate csv's: one-star, two-stars, and three-stars. For this project we need to combine all three csv's, delete and extract columns, and transform them into a database-friendly form.

To note: all restaurant data takes place in 2019.

In [2]:
# Modules
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy

%matplotlib inline

## Paths

In [3]:
michelin_path = "./Resources/michelin-restaurants/"

In [4]:
one_star = michelin_path + "one-star-michelin-restaurants.csv"
two_star = michelin_path + "two-stars-michelin-restaurants.csv"
three_star = michelin_path + "three-stars-michelin-restaurants.csv"

## Michelin Cleaning

Currently, the columns in each csv are:
- name
- year
- latitude
- longitude
- city
- region
- zipCode
- cuisine
- price
- url

`price`, `url`, `year`, and `zipCode` are superflous. We will be using an index named `michelin_id`to uniquely identify each restaurant. Once in a database, we need to allow searching for each restaurant based on a few attributes, namely `city` and `region`. In a specific region, we should add the flexibility to search for `cuisine` and `stars`, a new column to be appended to every restaurant.

In order to eliminate duplicates, `cuisine`, `city`, and `region` will be their own tables with unique identifiers. 

In [5]:
# Read the csv
one_df = pd.read_csv(one_star)
two_df = pd.read_csv(two_star)
three_df = pd.read_csv(three_star)

# Append 'stars' to the end of each table
one_df['stars'] = 1
two_df['stars'] = 2
three_df['stars'] = 3

# Combine csv's
michelin_df = pd.concat([one_df, two_df, three_df])
michelin_df

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
0,Kilian Stuba,2019,47.348580,10.171140,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...,1
1,Pfefferschiff,2019,47.837870,13.079170,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
2,Esszimmer,2019,47.806850,13.034090,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
3,Carpe Diem,2019,47.800010,13.040060,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
4,Edvard,2019,48.216503,16.368520,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,1
...,...,...,...,...,...,...,...,...,...,...,...
31,Fat Duck,2019,51.508280,-0.702320,Bray,United Kingdom,SL6 2AQ,Creative,NaN,https://guide.michelin.com/gb/en/buckinghamshi...,3
32,Waterside Inn,2019,51.507730,-0.701210,Bray,United Kingdom,SL6 2AT,Classic French,NaN,https://guide.michelin.com/gb/en/buckinghamshi...,3
33,Alain Ducasse at The Dorchester,2019,51.507120,-0.152520,Mayfair,United Kingdom,W1K 1QA,French,NaN,https://guide.michelin.com/gb/en/greater-londo...,3
34,The Araki,2019,51.511826,-0.140389,Mayfair,United Kingdom,W1S 3BF,Japanese,NaN,https://guide.michelin.com/gb/en/greater-londo...,3


In [6]:
# Drop superflous columns
michelin_df = michelin_df.drop(columns = ['year', 'url', 'price', 'zipCode'])

# Create and Rename index
michelin_df = michelin_df.reset_index(drop=True)
michelin_df = michelin_df.rename_axis("michelin_id")

# View Changes
michelin_df

,name,latitude,longitude,city,region,cuisine,stars
michelin_id,,,,,,,
0,Kilian Stuba,47.348580,10.171140,Kleinwalsertal,Austria,Creative,1
1,Pfefferschiff,47.837870,13.079170,Hallwang,Austria,Classic cuisine,1
2,Esszimmer,47.806850,13.034090,Salzburg,Austria,Creative,1
3,Carpe Diem,47.800010,13.040060,Salzburg,Austria,Market cuisine,1
4,Edvard,48.216503,16.368520,Wien,Austria,Modern cuisine,1
...,...,...,...,...,...,...,...
690,Fat Duck,51.508280,-0.702320,Bray,United Kingdom,Creative,3
691,Waterside Inn,51.507730,-0.701210,Bray,United Kingdom,Classic French,3
692,Alain Ducasse at The Dorchester,51.507120,-0.152520,Mayfair,United Kingdom,French,3


With the data combined and relatively cleaned, we need to extract the `cuisine`, `city`, and `region` keys and create seperate tables. This is easily done using the `unique()` method available to all DataFrames. 

In [7]:
# Create Cuisine DataFrame
cuisine_df = pd.DataFrame(michelin_df['cuisine'].unique())
cuisine_df = cuisine_df.rename(columns={0:"cuisine_name"})
cuisine_df['cuisine_id'] = numpy.arange(0,len(cuisine_df))
cuisine_df = cuisine_df[['cuisine_id', 'cuisine_name']]
cuisine_df.head()

,cuisine_id,cuisine_name
0,0,Creative
1,1,Classic cuisine
2,2,Market cuisine
3,3,Modern cuisine
4,4,Japanese


In [8]:
# Create Region DataFrame
region_df = pd.DataFrame(michelin_df['region'].unique())
region_df = region_df.rename(columns={0:'region_name'})
region_df['region_id'] = numpy.arange(0,len(region_df))
region_df = region_df[['region_id', 'region_name']]
region_df.head()

,region_id,region_name
0,0,Austria
1,1,California
2,2,Chicago
3,3,Croatia
4,4,Czech Republic


In [9]:
# Create City DataFrame
city_df = pd.DataFrame(michelin_df['city'].unique())
city_df = city_df.rename(columns={0:'city_name'})
city_df['city_id'] = numpy.arange(0, len(city_df))
city_df.head()

,city_name,city_id
0,Kleinwalsertal,0
1,Hallwang,1
2,Salzburg,2
3,Wien,3
4,South San Francisco,4


In [10]:
# Merge *_id tables
michelin_df = michelin_df.merge(city_df, left_on='city', right_on='city_name').sort_values(by='stars')
michelin_df = michelin_df.merge(region_df, left_on='region', right_on='region_name').sort_values(by='stars')
michelin_df = michelin_df.merge(cuisine_df, left_on='cuisine', right_on='cuisine_name').sort_values(by='stars')

michelin_df = michelin_df.drop(columns= ['city', 'city_name', 'region', 'region_name', 'cuisine', 'cuisine_name'])
michelin_df.head()

,name,latitude,longitude,stars,city_id,region_id,cuisine_id
0,Kilian Stuba,47.348580,10.171140,1,0,0,0
438,Impromptu by Paul Lee,25.054290,121.524254,1,64,20,33
439,PRU,8.034298,98.276400,1,66,21,33
440,Upstairs at Mikkeller,13.727946,100.588440,1,65,21,33
441,Gaa,13.737806,100.542750,1,65,21,33


In [11]:
# Clean latitude, longitude columns
def chop(flt):
    '''
    Truncate decimal to 5 decimal places
    '''
    return float("{.5f}".format(flt))

michelin_df['latitude'] = michelin_df['latitude'].apply(chop)
michelin_df['longitude'] = michelin_df['longitude'].apply(chop)

michelin_df.head()

AttributeError: 'float' object has no attribute '5f'

## CSV Creation